In [ ]:
import requests
from bs4 import BeautifulSoup

# Defino la base del sitio principal
BASE_URL = "https://books.toscrape.com/"

# Creo la URL inicial del sitio y uso BASE_URL + url_relativa para armar URLs completas de las categorías
CATEGORIAS_URL = BASE_URL + "index.html" 

# Extraigo con el método get las categorias y con el "parser" obtengo la info relevante
respuesta = requests.get(CATEGORIAS_URL)
soup = BeautifulSoup(respuesta.text, "html.parser")

# Extaigo los enlaces de categorias (hay uno para cada categoria dentro de 'ul.nav-list')
categorias_links = soup.select("ul.nav-list ul li a")

categoria = {} # Creo un diccionario vacio para guardar categorias

for link in categorias_links: 
    nombre = link.text.strip() # Obtengo y limpio el nombre de la categoria
    url_relativa = link['href'] # Saco la URL relativa del href
    url_completa = BASE_URL + url_relativa # Armo la URL completa
    categoria[nombre] = url_completa # Guardo en el diccionario: nombre -> URL

print(f"Categorias encontradas: {len(categoria)}\n")

# Recorre cada par del diccionario : (clave,valor) y con el .items devuelve la lista de pares
for nombre, url in categoria.items():
    print(f"- {nombre}: {url}")
    


